In [2]:
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
import yfinance as yf
from dataclasses import dataclass
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Validate environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")


@dataclass
class AnalysisConfig:
    """Configuration for analysis parameters"""

    period_years: int = 5
    interval: str = "1wk"  # 1d, 1wk, 1mo
    moving_averages: List[int] = None
    rsi_period: int = 14
    volume_ma_period: int = 20

    def __post_init__(self):
        if self.moving_averages is None:
            self.moving_averages = [50, 200]  # Default MAs


class FinancialMetrics:
    """Calculate financial metrics and ratios"""

    @staticmethod
    def calculate_ratios(financial_data: pd.DataFrame) -> Dict:
        try:
            current_ratio = (
                financial_data["Total Current Assets"]
                / financial_data["Total Current Liabilities"]
            )
            debt_to_equity = (
                financial_data["Total Liabilities"]
                / financial_data["Total Stockholder Equity"]
            )
            return {
                "current_ratio": current_ratio.iloc[-1],
                "debt_to_equity": debt_to_equity.iloc[-1],
                "quick_ratio": (
                    financial_data["Total Current Assets"] - financial_data["Inventory"]
                ).iloc[-1]
                / financial_data["Total Current Liabilities"].iloc[-1],
                "roe": (
                    financial_data["Net Income"]
                    / financial_data["Total Stockholder Equity"]
                ).iloc[-1],
            }
        except Exception as e:
            logger.error(f"Error calculating financial ratios: {e}")
            return {}


class TechnicalAnalysis:
    """Technical analysis calculations"""

    @staticmethod
    def calculate_rsi(data: pd.DataFrame, period: int = 14) -> pd.Series:
        delta = data["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def calculate_support_resistance(
        data: pd.DataFrame, window: int = 20
    ) -> Tuple[float, float]:
        rolling_min = data["Low"].rolling(window=window).min()
        rolling_max = data["High"].rolling(window=window).max()
        return rolling_min.iloc[-1], rolling_max.iloc[-1]


class ComprehensivePriceTool(BaseTool):
    name: str = "Comprehensive Price Analysis"
    description: str = "Get detailed historical price data with technical indicators"

    def __init__(self, config: AnalysisConfig):
        super().__init__()
        self._config = config  # Use _config instead of config

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(
                period=f"{self._config.period_years}y", interval=self._config.interval
            )

            if history.empty:
                return "No price data available for this ticker"

            # Calculate technical indicators
            for ma in self._config.moving_averages:
                history[f"MA_{ma}"] = history["Close"].rolling(window=ma).mean()

            history["RSI"] = TechnicalAnalysis.calculate_rsi(
                history, self._config.rsi_period
            )
            history["Volume_MA"] = (
                history["Volume"].rolling(window=self._config.volume_ma_period).mean()
            )

            support, resistance = TechnicalAnalysis.calculate_support_resistance(
                history
            )

            analysis_summary = {
                "price_data": history.to_csv(),
                "current_price": history["Close"].iloc[-1],
                "support_level": support,
                "resistance_level": resistance,
                "rsi": history["RSI"].iloc[-1],
                "volume_trend": (
                    "Up"
                    if history["Volume"].iloc[-1] > history["Volume_MA"].iloc[-1]
                    else "Down"
                ),
            }

            return str(analysis_summary)
        except Exception as e:
            logger.error(f"Error in price analysis: {e}")
            return f"Error analyzing price data: {str(e)}"


class FinancialMetricsTool(BaseTool):
    name: str = "Financial Metrics Analysis"
    description: str = "Analyze comprehensive financial metrics and growth trends"

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)

            # Get various financial statements
            balance_sheet = stock.balance_sheet
            income_stmt = stock.income_stmt
            cash_flow = stock.cashflow

            # Calculate key metrics and growth rates
            metrics = {
                "balance_sheet": balance_sheet.to_csv(),
                "income_statement": income_stmt.to_csv(),
                "cash_flow": cash_flow.to_csv(),
                "key_ratios": FinancialMetrics.calculate_ratios(balance_sheet),
                "revenue_growth": income_stmt.loc["Total Revenue"].pct_change().mean(),
                "profit_margin": (
                    income_stmt.loc["Net Income"] / income_stmt.loc["Total Revenue"]
                ).mean(),
            }

            return str(metrics)
        except Exception as e:
            logger.error(f"Error in financial analysis: {e}")
            return f"Error analyzing financial data: {str(e)}"


class AdvancedAgents:
    def __init__(self, config: AnalysisConfig):
        self.config = config

    def financial_analyst(self) -> Agent:
        return Agent(
            role="Senior Financial Analyst",
            goal="Provide comprehensive financial analysis focusing on long-term trends and company health",
            backstory="""A veteran financial analyst with 20 years of experience in equity research. 
                     Specialized in identifying long-term value opportunities through deep fundamental analysis.""",
            tools=[FinancialMetricsTool()],
            verbose=True,
        )

    def technical_analyst(self) -> Agent:
        return Agent(
            role="Technical Analysis Expert",
            goal="Analyze long-term price patterns and technical indicators to identify major trends",
            backstory="""A technical analysis expert with expertise in pattern recognition and trend analysis. 
                     Known for identifying major market turning points using multi-timeframe analysis.""",
            tools=[ComprehensivePriceTool(self.config)],
            verbose=True,
        )


def create_analysis_crew(ticker: str, config: AnalysisConfig = AnalysisConfig()) -> str:
    try:
        # Initialize agents
        agents = AdvancedAgents(config)

        # Create analysts
        financial_analyst = agents.financial_analyst()
        technical_analyst = agents.technical_analyst()

        # Define tasks with required expected_output field
        financial_analysis = Task(
            description=f"""Conduct a comprehensive {config.period_years}-year financial analysis of {ticker}.
                       Focus on: 
                       1. Long-term financial health trends
                       2. Growth metrics and sustainability
                       3. Risk factors and financial stability
                       4. Comparative industry analysis""",
            agent=financial_analyst,
            expected_output="""A detailed financial analysis report including:
                           - Key financial ratios and their trends
                           - Growth metrics and sustainability analysis
                           - Risk assessment
                           - Industry comparison metrics""",
        )

        technical_analysis = Task(
            description=f"""Analyze {config.period_years}-year price history for {ticker}.
                       Focus on:
                       1. Major trend identification
                       2. Support/resistance levels
                       3. Volume analysis
                       4. Technical indicator convergence/divergence""",
            agent=technical_analyst,
            expected_output="""A comprehensive technical analysis report including:
                           - Major trend analysis
                           - Support and resistance levels
                           - Volume analysis
                           - Technical indicator signals""",
        )

        # Create and configure crew
        crew = Crew(
            agents=[financial_analyst, technical_analyst],
            tasks=[financial_analysis, technical_analysis],
            verbose=True,
            process=Process.sequential,
            manager_llm=ChatOpenAI(
                model_name="gpt-4o-mini",
                temperature=0.5,
                api_key=OPENAI_API_KEY,
            ),
        )

        # Execute analysis
        result = crew.kickoff(inputs={"company": ticker})
        return result

    except Exception as e:
        logger.error(f"Error in analysis execution: {e}")
        raise


if __name__ == "__main__":
    # Example usage
    try:
        # Configure analysis parameters
        config = AnalysisConfig(
            period_years=5,
            interval="1wk",
            moving_averages=[50, 100, 200],
            rsi_period=14,
            volume_ma_period=20,
        )

        # Run analysis
        result = create_analysis_crew("066570.KS", config)
        print(result)

    except Exception as e:
        logger.error(f"Analysis failed: {e}")

ERROR:__main__:Error in analysis execution: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': 'Conduct ... fundamental analysis.)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
ERROR:__main__:Analysis failed: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': 'Conduct ... fundamental analysis.)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


LLM value is None
LLM value is None
